<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/IPA_practice01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

Note: Options to use

+ Lower case only
+ Voicing: voiced or voiceless
+ Place: bilabial, labio-dental, dental, alveolar, palato-alveolar, velar, gottal
+ Centrality: central or lateral
+ Manner: stop, fricative, affricate, approximant

In [ ]:
#@markdown IPA practice with points

import pandas as pd
import random
from IPython.display import display, clear_output
import ipywidgets as widgets

url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

# Load the CSV data
def load_data():
    return pd.read_csv(url, encoding='utf-8')

# Display a random IPA symbol and quiz the user
def quiz(df, score):
    clear_output(wait=True)
    row = df.sample(1).iloc[0]
    ipa_symbol = row['IPA']
    properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']
    question_property = random.choice(properties)

    print(f"IPA Symbol: {ipa_symbol}")
    print(f"What is the {question_property.lower()} of this IPA symbol?")

    input_text = widgets.Text(
        value='',
        placeholder='Type your answer here',
        description='Answer:',
        disabled=False
    )

    def check_answer(b):
        if input_text.value.lower() == str(row[question_property]).lower():
            score[0] += 1
            print(f"Correct! Total points: {score[0]}")
        else:
            print(f"Wrong! The correct answer was {row[question_property]}. Total points: {score[0]}")
        ask_continue(score)

    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(check_answer)

    display(input_text, submit_button)

# Ask whether to continue or quit the quiz
def ask_continue(score):
    continue_button = widgets.Button(description="Continue")
    quit_button = widgets.Button(description="Quit")

    def continue_quiz(b):
        quiz(df, score)

    def quit_quiz(b):
        clear_output(wait=True)
        print(f"Well-practiced! Your total points are: {score[0]}")

    continue_button.on_click(continue_quiz)
    quit_button.on_click(quit_quiz)

    display(continue_button, quit_button)

# Start the quiz
df = load_data()
score = [0]  # Using list to keep reference in nested functions
quiz(df, score)


# Graio

In [ ]:
!pip install gradio

In [ ]:
"https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"

In [ ]:
import pandas as pd
import random
import gradio as gr

# Load the CSV data from the provided URL
def load_data():
    url = "https://raw.githubusercontent.com/MK316/Myapps/main/data/IPA.csv"
    return pd.read_csv(url, encoding='utf-8')

df = load_data()  # Load data once at the start

# Generate a question from the data
def generate_question():
    row = df.sample(1).iloc[0]
    properties = ['Voicing', 'Place', 'Centrality', 'Oro-nasal', 'Manner']
    selected_property = random.choice(properties)
    ipa = row['IPA']
    answer = row[selected_property].lower()
    question = f"The {selected_property.lower()} of the IPA symbol [{ipa}]?"
    return question, answer

# Function to manage the quiz logic
def quiz_interface():
    with gr.Blocks() as app:
        score = [0]  # Using a list to hold the score which allows us to modify it in nested functions
        question_label = gr.Label()  # Displays the question
        answer_input = gr.Textbox(label="Type your answer here")  # For user to input their answer
        result_label = gr.Label()  # To display feedback
        continue_button = gr.Button("Continue")
        quit_button = gr.Button("Quit")

        # Function to update the question and check the answer
        def check_answer(answer):
            correct_answer = question_label.metadata['answer']
            if answer.lower() == correct_answer:
                score[0] += 1
                result_label.value = f"Correct! You've got {score[0]} point(s)."
            else:
                result_label.value = f"Wrong! The correct answer was {correct_answer}."
            # Prepare the next question
            next_question, next_answer = generate_question()
            question_label.value = next_question
            question_label.metadata['answer'] = next_answer
            answer_input.value = ""  # Clear the input box

        # Continue without checking the answer
        def continue_quiz():
            next_question, next_answer = generate_question()
            question_label.value = next_question
            question_label.metadata['answer'] = next_answer
            answer_input.value = ""
            result_label.value = ""

        # Ending the quiz
        def quit_quiz():
            result_label.value = f"Quiz ended. Your total score: {score[0]} points."
            answer_input.disabled = True
            continue_button.disabled = True
            quit_button.disabled = True

        # Bind functions to buttons
        answer_input.submit(check_answer)
        continue_button.click(continue_quiz)
        quit_button.click(quit_quiz)

        # Add components to the interface
        app.add(question_label)
        app.add(answer_input)
        app.add(result_label)
        app.add(continue_button)
        app.add(quit_button)

        # Initialize the first question
        first_question, first_answer = generate_question()
        question_label.value = first_question
        question_label.metadata = {'answer': first_answer}  # Store the answer in the label metadata for access

    return app

# Create and launch the app
app = quiz_interface()
app.launch()
